In [1]:
import re
import time
import datetime
import requests
import pandas as pd
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from Script.Function.funciones import movies_for_category, categories, print_process, movies, series_informacion, desempaquetado, generador_dummies

# -------------- Parametros --------------
ruta_base = r"Bases_datos/"

# ----------------------------------------

In [2]:
print_process("Comienzo del Notebook 'Scraping'")

|-----------------------Comienzo del Notebook 'Scraping'-----------------------|


In [3]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 27.2 segundos-----------------------|


In [4]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 26.86 segundos|
|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 27.4 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 28.65 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 29.04 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 29.49 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 29.84 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 31.83 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 32.73 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 42.46 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 14.35 segundos|
|Todo el contenido de la categ

In [5]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [6]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

|-----Comienzo del proceso de extraccion de informacion de cada iteracion-----|


In [7]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies) / 60,2)} minutos")


Error al cargar la url https://filmzie.com/content/the-wedding en el intento 1: 404


In [ ]:
df = pd.DataFrame(todo)

In [ ]:

df.to_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [2]:
df = pd.read_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv")

In [3]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [4]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)
peliculas.loc[peliculas["idioma"] == 'Non linguistic content', "idioma"] = "No contiene idioma"
peliculas["subtitulos"] = peliculas["subtitulos"].fillna("No contiene subtitulos")


In [5]:
peliculas.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,The Last Schnitzel,Turkish,English (embedded),2017,22,"Short, Comedy, Fantasy, Sci-Fi, European Produ...","In a future where Earth is uninhabitable, the ...",Pelicula,https://filmzie.com/content/the-last-schnitzel
1,The Peacekeepers,Multiple languages,No contiene subtitulos,2005,83,Documentary,"""The Peacekeepers"" grants rare access to Unite...",Pelicula,https://filmzie.com/content/the-peacekeepers
2,Kanehsatake: 270 Years of Resistance,English,No contiene subtitulos,1993,119,Documentary,Witness the Mohawk Nation's resistance to cent...,Pelicula,https://filmzie.com/content/kanehsatake-270-ye...
3,The Bead Game,English,No contiene subtitulos,1977,5,"Animation, Short, Comedy, Family, Fantasy","Accompanied by rhythmic beats, evolving creatu...",Pelicula,https://filmzie.com/content/the-bead-game
4,U,English,No contiene subtitulos,2022,60,Drama,Embark on a rollercoaster ride through four in...,Pelicula,https://filmzie.com/content/u-2022


In [6]:
peliculas.to_csv(f"{ruta_base}Peliculas.csv",index=False)

In [7]:
peliculas_dummies = generador_dummies(peliculas)

In [8]:
# Exportamos base de datos de Peliculas con dummies
peliculas_dummies.to_csv(f"{ruta_base}Peliculas_dummies.csv",index=False)

peliculas_dummies.head()

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,Action,Crime,...,Animation,Romance,Music,War,Mystery,Documentary,Thriller,Musical,Comedy,Short
0,The Last Schnitzel,Turkish,English (embedded),2017,22,"In a future where Earth is uninhabitable, the ...",Pelicula,https://filmzie.com/content/the-last-schnitzel,0,0,...,0,0,0,0,0,0,0,0,1,1
1,The Peacekeepers,Multiple languages,No contiene subtitulos,2005,83,"""The Peacekeepers"" grants rare access to Unite...",Pelicula,https://filmzie.com/content/the-peacekeepers,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Kanehsatake: 270 Years of Resistance,English,No contiene subtitulos,1993,119,Witness the Mohawk Nation's resistance to cent...,Pelicula,https://filmzie.com/content/kanehsatake-270-ye...,0,0,...,0,0,0,0,0,1,0,0,0,0
3,The Bead Game,English,No contiene subtitulos,1977,5,"Accompanied by rhythmic beats, evolving creatu...",Pelicula,https://filmzie.com/content/the-bead-game,0,0,...,1,0,0,0,0,0,0,0,1,1
4,U,English,No contiene subtitulos,2022,60,Embark on a rollercoaster ride through four in...,Pelicula,https://filmzie.com/content/u-2022,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
series.drop(columns=['duracion_minutos'], inplace=True)
series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,Kingdom and Wisdom's Parables,English,English,2019,Animation,Stories with important messages addressing per...,Serie,https://filmzie.com/content/kingdom-and-wisdom...
1,"Wow, I Never Knew That!",English,English,2011,History,Discover the intriguing origins of everyday th...,Serie,https://filmzie.com/content/wow-i-never-knew-t...
2,LARPs,English,English,2014,"Comedy, Drama",Six friends discover how their real lives affe...,Serie,https://filmzie.com/content/larps-2014
3,The Benza,Japanese,English,2019,"Comedy, Adventure","Chris and Kyle share a house in Tokyo, Japan. ...",Serie,https://filmzie.com/content/the-benza-2019
4,The Farm,English,English,2013,Reality TV,"""The Farm"" beckons you to join the farm-to-tab...",Serie,https://filmzie.com/content/the-farm-2013
...,...,...,...,...,...,...,...,...
94,In Search of Walt Whitman,English,English,2020,Documentary,Walt Whitman rises from a hardscrabble boyhood...,Serie,https://filmzie.com/content/in-search-of-walt-...
95,The Ave,English,English,2015,Drama,Young entrepreneurs try to rise above the grit...,Serie,https://filmzie.com/content/the-ave-2015
96,Scenic Nature: Seasons,English,NaN,2023,Documentary,Relax with the Seasons of Nature in 4K UHD res...,Serie,https://filmzie.com/content/scenic-nature-seas...
97,Chews Me,English,English,2022,Reality TV,"A culinary blind date: 3 singles, 1 kitchen, 1...",Serie,https://filmzie.com/content/chews-me-2022


In [10]:
links_series = list(series["link"].values)
links_series

['https://filmzie.com/content/kingdom-and-wisdoms-parables-2019',
 'https://filmzie.com/content/wow-i-never-knew-that-2011',
 'https://filmzie.com/content/larps-2014',
 'https://filmzie.com/content/the-benza-2019',
 'https://filmzie.com/content/the-farm-2013',
 'https://filmzie.com/content/2nd-generation-2018',
 'https://filmzie.com/content/bulge-bracket-2020',
 'https://filmzie.com/content/backyard-kitchen-2020',
 'https://filmzie.com/content/wonder-world-of-science-1989',
 'https://filmzie.com/content/the-shadow-dimension-2021',
 'https://filmzie.com/content/ghost-dimension-2016',
 'https://filmzie.com/content/southwest-living-2006',
 'https://filmzie.com/content/journey-to-a-new-earth-2022',
 'https://filmzie.com/content/luxury-travel-show-2018',
 'https://filmzie.com/content/the-special-seer-2019',
 'https://filmzie.com/content/threesome-2015',
 'https://filmzie.com/content/chemistry-series-1983',
 'https://filmzie.com/content/the-psychic-2019',
 'https://filmzie.com/content/webers

In [11]:
procesar = links_series[:3]

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": [], "sinopsis" : [], "duracion_minutos" : []}
# diccionario = {"titulo" : alm_3[0], "temporada" : alm_3[1], "capitulos" : alm_3[2], "sinopsis" : alm_3[3], "duracion_minutos" : alm_3[4]}
series_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, procesar))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos, sinopsis, duracion_minutos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)
resultados_dict["sinopsis"].extend(sinopsis)
resultados_dict["duracion_minutos"].extend(duracion_minutos)

series_fin = time.time()

print_process(f"La extraccion total de informacion de cada serie tuvo una demora de {round((series_fin-series_comienzo)/ 60, 2) } minutos")

Tiempo total de Kingdom and Wisdom's Parables: 19.33 segundos
Tiempo total de Wow, I Never Knew That!: 19.35 segundos
Tiempo total de LARPs: 23.25 segundos
|La extraccion total de informacion de cada serie tuvo una demora de 0.46 minutos|


In [12]:
df_2 = pd.DataFrame(desempaquetado(resultados_dict))
df_2.head()

,titulo,temporadas,capitulos,sinopsis,duracion_minutos


In [13]:
series_descripcion = pd.merge(series,df_2, on="titulo", how="inner")
series_descripcion.head()

,titulo,idioma,subtitulos,ano,categorias,sinopsis_x,tipo,link,temporadas,capitulos,sinopsis_y,duracion_minutos
